# Linear Algebra

In [37]:
import iarray as ia
import numpy as np

### Matrix-Vector multiplication

*ironArray* also allows users to perform matrix-vector multiplication in the same way that
matrix-matrix multiplication is performed.

As said in the previous tutorial, if we want to set some parameters in the out array (for example, the chunkshape and the blockshape), we must use the `ia.matmul` function to perform the multiplication.

In [38]:
filename = "examples/arr-gemv.iarr"
ia.remove_urlpath(filename)
avshape = (3000, 5000)
av = ia.random.normal(avshape, 3, 2, chunks=(1000, 1000), blocks=(250, 250), urlpath=filename, fp_mantissa_bits=20)

bvshape = (5000,)
bv = ia.ones(bvshape, chunks=(1000,), blocks=(250,))

In [39]:
%%mprof_run iarray::mvmul
cv = ia.matmul(av, bv, chunks=(1000,), blocks=(200,))
print(cv.info)

type   : IArray
shape  : (3000,)
chunks : (1000,)
blocks : (200,)
cratio : 1.28

memprofiler: used 38.21 MiB RAM (peak of 38.23 MiB) in 0.0883 s, total RAM usage 337.76 MiB


Now, the output array has the chunkshape and the blockshape that we have specified in the `ia.matmul` function.

### Optimization Tips

As well as in the matrix-matrix multiplication, *ironArray* also has an `ia.matmul_gemv_params` function to calculate good chunk and block shapes for the operands in a matrix-vector multiplication. The use of it is pretty similar to the `ia.matmul_gemm_params` function, with the main difference that we only have to pass the shape from the first operand.


In [40]:
vparams = ia.matmul_gemv_params(avshape[0], avshape[1], itemsize=8)
avchunks, avblocks, bvchunks, bvblocks = vparams
ia.remove_urlpath(filename)

ashape = (30000, 5000)
av = ia.random.normal(avshape, 3, 2, chunks=avchunks, blocks=avblocks, urlpath=filename, fp_mantissa_bits=20)
print(av.info)

bvshape = (5000,)
bv = ia.ones(bvshape, chunks=bvchunks, blocks=bvblocks)
print(bv.info)


type   : IArray
shape  : (3000, 5000)
chunks : (3060, 4335)
blocks : (255, 255)
cratio : 2.36

type   : IArray
shape  : (5000,)
chunks : (4335,)
blocks : (255,)
cratio : 481.67



In [41]:
%%mprof_run iarray::op_mvmul
cv = ia.matmul(av, bv)
print(cv.info)

type   : IArray
shape  : (3000,)
chunks : (3060,)
blocks : (255,)
cratio : 1.31

memprofiler: used -21.38 MiB RAM (peak of 1.59 MiB) in 0.0322 s, total RAM usage 555.58 MiB


In [42]:
%mprof_plot iarray::.*mvmul  -t "Matrix-Vector multiplication"

This time, with the recommended chunk and block shapes we consume less memory and spent less time computing the multiplication. Let's compare it now with NumPy and Zarr+Dask.

In [43]:
import zarr
# Set up the arrays
zfilename = "examples/arr-gemv.zarr"
ia.remove_urlpath(zfilename)

zav = zarr.open(zfilename, mode='w', shape=avshape)
av.copyto(zav)
print(zav.info)

zbv = zarr.create(shape=bvshape)
bv.copyto(zbv)
print(zbv.info)


Type               : zarr.core.Array
Data type          : float64
Shape              : (3000, 5000)
Chunk shape        : (188, 625)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : zarr.storage.DirectoryStore
No. bytes          : 120000000 (114.4M)
No. bytes stored   : 52518345 (50.1M)
Storage ratio      : 2.3
Chunks initialized : 128/128

Type               : zarr.core.Array
Data type          : float64
Shape              : (5000,)
Chunk shape        : (5000,)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : builtins.dict
No. bytes          : 40000 (39.1K)
No. bytes stored   : 609
Storage ratio      : 65.7
Chunks initialized : 1/1



First we compute the NumPy multiplication:

In [44]:
%%mprof_run numpy::mvmul

npcv = np.matmul(av.data, bv.data)

memprofiler: used 112.80 MiB RAM (peak of 114.48 MiB) in 0.1361 s, total RAM usage 572.91 MiB


And now it is the turn of Zarr+Dask:

In [45]:
import dask.array as da

In [46]:
%%mprof_run dask::mvmul
res = da.matmul(zav, zbv)
cshape = (zav.shape[0], )
cchunks = (zav.chunks[0], )
cv = zarr.create(shape=cshape)
da.to_zarr(res, cv)

print(cv.info)



Type               : zarr.core.Array
Data type          : float64
Shape              : (3000,)
Chunk shape        : (3000,)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : builtins.dict
No. bytes          : 24000 (23.4K)
No. bytes stored   : 18221 (17.8K)
Storage ratio      : 1.3
Chunks initialized : 1/1

memprofiler: used -0.49 MiB RAM (peak of 0.00 MiB) in 0.1453 s, total RAM usage 572.42 MiB


In [47]:
%mprof_plot iarray::op_mvmul numpy::mvmul dask::mvmul -t "Matrix-Vector computation"

This time, without even favoring the speed, *ironArray* obtains better results when it comes to memory usage and computation time.
